## Instalación de dependencias

In [ ]:
# Instala memory-profiler en Colab
%pip install memory-profiler

# Necesario para mostrar el gráfico de uso de memoria en Colab
%pip install matplotlib

%pip install memory_profiler

# Preparación del archivo como antes
%pip install gdown
url = 'https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis'
!gdown "$url" -O "archivo.zip"
!unzip -o "archivo.zip" -d "./"

## Implementación de la función Q1 Time

In [8]:
import json
from collections import defaultdict, Counter
from datetime import datetime
from typing import List, Tuple, Optional

def map_function(line: str) -> Optional[Tuple[datetime.date, str]]:
    try:
        tweet = json.loads(line)
        fecha = datetime.strptime(tweet['date'][:10], '%Y-%m-%d').date()
        usuario = tweet['user']['username']
        return fecha, usuario
    except json.JSONDecodeError as e:
        print(f"Error decodificando JSON: {e}")
        return None

def reduce_function(mapped_values: List[Tuple[datetime.date, str]]) -> List[Tuple[datetime.date, str]]:
    tweets_por_fecha = defaultdict(list)
    for value in mapped_values:
        if value is not None:
            fecha, usuario = value
            tweets_por_fecha[fecha].append(usuario)

    usuarios_top_por_fecha = []
    for fecha, usuarios in tweets_por_fecha.items():
        conteo_usuarios = Counter(usuarios)
        usuario_top = conteo_usuarios.most_common(1)[0][0]
        usuarios_top_por_fecha.append((fecha, usuario_top))

    return sorted(usuarios_top_por_fecha, key=lambda x: len(tweets_por_fecha[x[0]]), reverse=True)[:10]

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    mapped_values = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            result = map_function(line)
            if result is not None:
                mapped_values.append(result)

    top_fechas = reduce_function(mapped_values)
    return top_fechas

archivo_json = './farmers-protest-tweets-2021-2-4.json'
resultado = q1_time(archivo_json)
print(resultado)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


# Medición del consumo de memoria y tiempo de ejecución de la función Q1 Time


Resultado de medición expuesto por medio de un archivo jpg

In [9]:
import json
from collections import defaultdict, Counter
from datetime import datetime
import cProfile, pstats
from memory_profiler import memory_usage
import matplotlib.pyplot as plt

def map_function(line: str) -> Optional[Tuple[datetime.date, str]]:
    try:
        tweet = json.loads(line)
        fecha = datetime.strptime(tweet['date'][:10], '%Y-%m-%d').date()
        usuario = tweet['user']['username']
        return fecha, usuario
    except json.JSONDecodeError as e:
        print(f"Error decodificando JSON: {e}")
        return None

def reduce_function(mapped_values: List[Tuple[datetime.date, str]]) -> List[Tuple[datetime.date, str]]:
    tweets_por_fecha = defaultdict(list)
    for value in mapped_values:
        if value is not None:
            fecha, usuario = value
            tweets_por_fecha[fecha].append(usuario)

    usuarios_top_por_fecha = []
    for fecha, usuarios in tweets_por_fecha.items():
        conteo_usuarios = Counter(usuarios)
        usuario_top = conteo_usuarios.most_common(1)[0][0]
        usuarios_top_por_fecha.append((fecha, usuario_top))

    return sorted(usuarios_top_por_fecha, key=lambda x: len(tweets_por_fecha[x[0]]), reverse=True)[:10]

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    mapped_values = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            result = map_function(line)
            if result is not None:
                mapped_values.append(result)

    top_fechas = reduce_function(mapped_values)
    return top_fechas


def profile_memory_and_cpu():
    archivo_json = './farmers-protest-tweets-2021-2-4.json'

    # Profile de memoria
    mem_usage = memory_usage((q1_time, (archivo_json,)), interval=0.1)

    # Profile de tiempo de CPU
    profiler = cProfile.Profile()
    profiler.enable()
    q1_time(archivo_json)
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('cumtime')
    stats.print_stats()

    # Plot de uso de memoria y guardar en archivo .jpg
    plt.figure(figsize=(10, 6))
    plt.plot(mem_usage)
    plt.title('Uso de memoria a lo largo del tiempo')
    plt.xlabel('Tiempo (en intervalos de 0.1s)')
    plt.ylabel('Memoria usada (MB)')
    plt.savefig('q1_memory_resources_map_reduce.jpg')
    plt.close()

profile_memory_and_cpu()

         4561201 function calls in 9.322 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.634    1.634    9.322    9.322 <ipython-input-9-7198e32df3b1>:33(q1_time)
   117407    0.272    0.000    7.320    0.000 <ipython-input-9-7198e32df3b1>:8(map_function)
   117407    0.193    0.000    4.506    0.000 /usr/lib/python3.10/json/__init__.py:299(loads)
   117407    0.292    0.000    4.247    0.000 /usr/lib/python3.10/json/decoder.py:332(decode)
   117407    3.755    0.000    3.755    0.000 /usr/lib/python3.10/json/decoder.py:343(raw_decode)
   117407    0.108    0.000    2.520    0.000 {built-in method strptime}
   117407    0.219    0.000    2.412    0.000 /usr/lib/python3.10/_strptime.py:565(_strptime_datetime)
   117407    1.247    0.000    2.193    0.000 /usr/lib/python3.10/_strptime.py:309(_strptime)
   117407    0.072    0.000    0.533    0.000 /usr/lib/python3.10/_strptime.py:26(_getlang)
   117407    0.0